In [1]:
import numpy as np

G = np.array([[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1],
              [0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
              [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
              [0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
              [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1],
              [0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]])

y = np.array([1, 1, 0, 1, 0, 0 ,0 ,0 ,1 ,1 , 0, 0, 1, 0, 1, 0])

In [2]:
def perm_row(row, inds):
    return np.array([row[i] for i in inds])

def perm_matrix(matrix, inds):
    return np.array([perm_row(matrix[i], inds) for i in range(len(matrix))])

def move_rows(G, i):
    for j in range(i, len(G)):
        if G[j][i] != 0:
            tmp = np.copy(G[j])
            G[j] = G[i]
            G[i] = tmp
            return

def gauss(G):
    r = len(G)
    n = len(G[0])
    for i in range(r):
        if G[i][i] == 0:
            move_rows(G, i)
        if G[i][i] == 0:
            continue
        for j in range(r):
            if i == j: 
                continue
            coef = G[j][i] / G[i][i]
            for k in range(n):
                G[j][k] = (G[j][k] + 2 - coef * G[i][k]) % 2
                
def mod2dot(x, G):
    return np.array(x).dot(G) % 2

def rev_perm_row(row, inds):
    return [row[list(inds).index(i)] for i in range(len(row))]

def rev_perm_matrix(matrix, inds):
    return np.array([rev_perm_row(matrix[i], inds) for i in range(len(matrix))])

def get_err_v(msg, code):
    err_v = []
    for i in range(len(code)):
        if msg[i] != code[i]:
            err_v.append(i)
    return err_v

In [3]:
def subtr(g, i, j):
    for ind in range(len(i)):
        cur_g[i[ind]] = (cur_g[i[ind]] + 2 - cur_g[j]) % 2

def find_st_fn(row):
    st, fn = (-1, -1)
    was1 = False
    for ind in range(len(row)):
        el = row[ind]
        if el == 1:
            if was1:
                fn = ind
            else:
                was1 = True
                st = ind
    return (st, fn)

print(G)
cur_g = np.copy(G)
gauss(cur_g)
# Да, руками
subtr(cur_g, [2], 6)
subtr(cur_g, [0, 1, 2, 3], 5)
subtr(cur_g, [0, 3], 4)
subtr(cur_g, [0, 2], 3)
subtr(cur_g, [0], 1)
print()
print(cur_g)
sts = []
fns = []
for ind in range(len(cur_g)):
    st, fn = find_st_fn(cur_g[ind])
    sts.append(st)
    fns.append(fn)
    print("{}: st {}, fn {}".format(ind, st, fn))

[[1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 1]
 [0 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0]
 [0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1]
 [0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0]
 [0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1]
 [0 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1]]

[[1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0]]
0: st 0, fn 5
1: st 1, fn 8
2: st 2, fn 4
3: st 3, fn 9
4: st 4, fn 10
5: st 5, fn 13
6: st 10, fn 15
7: st 9, fn 14


In [4]:
import itertools as it
from math import log2

def all_bin(n):
    return np.array(list(map(list, it.product([False, True], repeat=n))))

class Node:
    def __init__(self, id, active_rows):
        self.id = id
        self.active_rows = active_rows
        self.edges = []
    def add_edge(self, node, w):
        self.edges.append((node, w))

def intToMask(n, size):
    res = []
    for _ in range(size):
        res.append(n % 2 == 0)
        n //= 2
    return np.array(res)
        

def buildGraph(G, sts, fns):
    k, n = G.shape
    current_active_rows = np.zeros(k, dtype=bool)
    layers = [[Node(0, current_active_rows.copy())]]

    active_rows = [current_active_rows.copy()]
    active_rows_cnt = 0
    id = 1
    for i in range(n):
        if i in sts:
            current_active_rows[sts.index(i)] = True
            active_rows_cnt += 1
        if i in fns:
            current_active_rows[fns.index(i)] = False
            active_rows_cnt -= 1
        layer = []
        allMasks = all_bin(active_rows_cnt)
        for mask in allMasks:
            node_active_rows = np.zeros(k, dtype=bool)
            node_active_rows[current_active_rows] = mask
            layer.append(Node(id, node_active_rows))
            id += 1
        layers.append(layer)
        active_rows.append(current_active_rows.copy())

    for i in range(n):
        for node in layers[i]:
            common = (active_rows[i] & active_rows[i + 1])
            for nextNode in layers[i + 1]:
                if ((nextNode.active_rows & common) == (node.active_rows & common)).all():
                    node_active_rows = nextNode.active_rows | node.active_rows
                    w = mod2dot(node_active_rows, G[:, i])
                    node.add_edge(nextNode, w)
    
    return layers, active_rows, id

In [5]:
from collections import deque

def to_str_row(row):
    return "".join(map(str, row))

def decode(G, y, sts, fns):
    print("Min_span:")
    for row in G:
        print(to_str_row(row))
    layers, current_active_rows, last_id = buildGraph(G, sts, fns)
    print(*map(lambda l : int(np.log2(len(l))), layers))
    k, n = G.shape
    ds = [1e9 for _ in range(last_id)]
    prev = [(None, -1) for _ in range(last_id)]
    ds[0] = 0
    q = deque()
    q.append((layers[0][0], 0))
    while len(q) != 0:
        node, layer_num = q.popleft()
        d = ds[node.id]

        for (next_node, w) in node.edges:
            if ds[next_node.id] > d + int(y[layer_num] != w):
                ds[next_node.id] = d + int(y[layer_num] != w)
                prev[next_node.id] = (node, w)
                q.append((next_node, layer_num + 1))

    cur = layers[len(layers) - 1][0]
    res = []
    edges = set()
    while cur is not None:
        nxt, w = prev[cur.id]
        if w != -1:
            res.append(w)
            edges.add((nxt.id, cur.id))
        cur = nxt
        
    res = np.array(res[::-1])
    
    print("Error_locations:")
    print(*(np.where(y != res)[0]))
    print("Real:     {}".format(to_str_row(y)))
    print("Expected: {}".format(to_str_row(res)))

    return edges

In [6]:
edges = decode(cur_g, y, sts, fns)

Min_span:
1101010000000000
0100011110000000
0011100000000000
0001110101000000
0000100001100000
0000010010110100
0000000000101001
0000000001111010
0 1 2 3 4 4 4 4 4 3 3 3 3 3 2 1 0
Error_locations:
13
Real:     1101000011001010
Expected: 1101000011001110


In [1]:
import graphviz as gv


def boolsToStr(bools):
    return "".join(map(str, bools.astype(int)))

def printGraph(G, edges):
    gr = gv.Digraph()
    gr.attr(rankdir="LR", shape='circle', ranksep='3.5')
    kek = np.arange(G.shape[0])
    G = cur_g
    layers, current_active_rows, _ = buildGraph(G, sts, fns)
    for i in range(len(layers)):
        active = kek[current_active_rows[i]]
        with gr.subgraph() as layer:
            layer.attr(rank='same', rankdir= "TB")
            layer.attr(label=f"{kek[current_active_rows[i]]}")
            for node in layers[i]:
                state = node.active_rows[current_active_rows[i]]
                nodeId = f"{i}_{boolsToStr(state)}"
                layer.node(nodeId, label=boolsToStr(state))
                for c in range(len(node.edges)):
                    (nextNode, w) = node.edges[c]
                    nextNodeId = f"{i + 1}_{boolsToStr(nextNode.active_rows[current_active_rows[i + 1]])}"
                    if (node.id, nextNode.id) in edges:
                        color = "red"
                    else:
                        color = "black"
                    gr.edge(nodeId, nextNodeId, label=f'{w}', color=color, fontcolor=color)
    return gr
gr = printGraph(G, edges)

NameError: name 'G' is not defined

In [8]:
gr.view()

'Digraph.gv.pdf'